In [108]:
import pandas as pd

model_csv = 'C:/Users/usuario/Desktop/Photo_Categorization/photoCategorization/data/results3/blip2_t5_pretrain_flant5xl_3187.csv'
df = pd.read_csv(model_csv)

In [109]:
conteo_paises = df['Country'].value_counts()
num_paises_unicos = conteo_paises.nunique()

In [110]:
print(f"Total de países distintos: {num_paises_unicos}")
print(conteo_paises)

Total de países distintos: 5
Country
ESPAÑA     61
TURQUÍA    10
EGIPTO      9
CHIPRE      7
TURQUIA     7
MALTA       2
Name: count, dtype: int64


In [111]:
conteo_paises = df['Region'].value_counts()
num_paises_unicos = conteo_paises.nunique()

In [112]:
print(f"Total de países distintos: {num_paises_unicos}")
print(conteo_paises)

Total de países distintos: 9
Region
TÜRKISCHERIVIERA    15
GRANCANARIA         13
MALLORCA            13
LANZAROTE           12
TENERIFE            11
FUERTEVENTURA       10
EGIPTO               9
OTROS                8
TÜRKISCHEÄGÄIS       2
FUERTEVETURA         2
WESTKÜSTE            1
Name: count, dtype: int64


### Prueba de clasificación de eventuales

In [113]:
df_yes = df[df['People'].str.contains('yes', case=False, na=False)]

In [114]:
porcentajes = df_yes.groupby('Country').size() / df.groupby('Country').size() * 100
porcentajes = porcentajes.fillna(0)
print(porcentajes)
valor_min = porcentajes.min()
valor_max = porcentajes.max()

Country
CHIPRE     14.285714
EGIPTO     33.333333
ESPAÑA     26.229508
MALTA       0.000000
TURQUIA    28.571429
TURQUÍA    10.000000
dtype: float64


In [115]:
valores_normalizados = (porcentajes - valor_min) / (valor_max - valor_min)
print(valores_normalizados)

Country
CHIPRE     0.428571
EGIPTO     1.000000
ESPAÑA     0.786885
MALTA      0.000000
TURQUIA    0.857143
TURQUÍA    0.300000
dtype: float64


In [116]:
porcentajes = df_yes.groupby('Region').size() / df.groupby('Region').size() * 100
porcentajes = porcentajes.fillna(0)
print(porcentajes)
valor_min = porcentajes.min()
valor_max = porcentajes.max()

Region
EGIPTO              33.333333
FUERTEVENTURA       40.000000
FUERTEVETURA         0.000000
GRANCANARIA         23.076923
LANZAROTE            8.333333
MALLORCA            53.846154
OTROS               12.500000
TENERIFE             9.090909
TÜRKISCHERIVIERA    20.000000
TÜRKISCHEÄGÄIS       0.000000
WESTKÜSTE            0.000000
dtype: float64


In [117]:
valores_normalizados = (porcentajes - valor_min) / (valor_max - valor_min)
print(valores_normalizados)

Region
EGIPTO              0.619048
FUERTEVENTURA       0.742857
FUERTEVETURA        0.000000
GRANCANARIA         0.428571
LANZAROTE           0.154762
MALLORCA            1.000000
OTROS               0.232143
TENERIFE            0.168831
TÜRKISCHERIVIERA    0.371429
TÜRKISCHEÄGÄIS      0.000000
WESTKÜSTE           0.000000
dtype: float64


### Intento de automatización del proceso

In [118]:
import pandas as pd

#Creación de una opcion de automatizado por medio de indexión de tabla

model_csv = 'C:/Users/usuario/Desktop/Photo_Categorization/photoCategorization/data/results3/blip2_t5_pretrain_flant5xl_3187.csv'
df = pd.read_csv(model_csv)

In [119]:
#Falta termina la adición de las ultimas columnas 

condiciones = {
    'People': ['yes'],  
    'Role': ['primary', 'secondary'],          
    'Gender': ['male', 'female', 'other'],
    'Category': ['tourists', 'locals', 'workers'],
    'Age_Categories': ['children', 'young adults', 'adults', 'elderly'],
    'Travel_Group': ['family', 'couple'],
    'Activity_Type': ['active', 'passive'],
    'Context_Type': ['tourist attraction', 'leisure and recreation', 'lodging facility', 'natural resource', 'infrastructure'],
    'Resource_Types': ['flora', 'fauna', 'landscapes'],    
}

# Crear un DataFrame de combinaciones columna-variable
combinaciones = pd.DataFrame([
    {'Columna': col, 'Variable': var}
    for col, vars_ in condiciones.items()
    for var in vars_
])

In [120]:
def calcular_normalizado(df, columna, variable):
    # Filtrar filas que contienen la variable actual
    df_filtrado = df[df[columna].str.contains(variable, case=False, na=False)]

    # Calcular porcentajes por país
    porcentajes = df_filtrado.groupby('Country').size() / df.groupby('Country').size() * 100
    #porcentajes = df_filtrado.groupby('Region').size() / df.groupby('Region').size() * 100
    porcentajes = porcentajes.fillna(0)

    # Obtener los valores mínimo y máximo globales para normalizar
    valor_min = porcentajes.min()
    valor_max = porcentajes.max()

    # Calcular valores normalizados
    valores_normalizados = (porcentajes - valor_min) / (valor_max - valor_min)

    return porcentajes, valores_normalizados

In [121]:
def graficar_resultados_multiple(resultados_list):
    sns.set(style="whitegrid")
    num_combinaciones = len(resultados_list)
    num_paginas = (num_combinaciones // 3) + (num_combinaciones % 3 > 0)  # Calcular el número de páginas

    for page in range(num_paginas):
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        fig.suptitle(f"Página {page + 1}", fontsize=16)

        for i in range(3):
            idx = page * 3 + i
            if idx >= num_combinaciones:
                axes[i].axis('off')  # Apagar el subplot vacío
                continue

            # Datos para la combinación actual
            columna, variable, porcentajes, valores_normalizados = resultados_list[idx]
            df_grafico = pd.DataFrame({
                'País': porcentajes.index,
                'Porcentaje': porcentajes.values,
                'Valor Normalizado': valores_normalizados.values
            }).sort_values(by='Valor Normalizado', ascending=False)

            # Crear gráfico
            sns.barplot(data=df_grafico, x='País', y='Valor Normalizado', ax=axes[i], palette="coolwarm", hue='País', dodge=False)
            axes[i].set_title(f"{variable} en {columna}")
            axes[i].set_xlabel("País")
            axes[i].set_ylabel("Valor Normalizado")
            axes[i].tick_params(axis='x', rotation=45)
            axes[i].legend([], frameon=False)  # Ocultar leyenda automática

        plt.tight_layout()
        plt.subplots_adjust(top=0.85)  # Ajustar espacio para el título principal
        plt.show()

In [122]:
def formatear_resultados(resultados_list):
    df_formateado = pd.DataFrame()

    # Crear un DataFrame para cada combinación
    for columna, variable, porcentajes, valores_normalizados in resultados_list:
        df_temp = pd.DataFrame({
            'País': porcentajes.index,
            'Valor Normalizado': valores_normalizados.values
        })
        df_temp['Índice'] = f"{columna} - {variable}"
        df_temp = df_temp.pivot(index='Índice', columns='País', values='Valor Normalizado')
        df_formateado = pd.concat([df_formateado, df_temp])

    return df_formateado

In [123]:
resultados_list = []
for _, row in combinaciones.iterrows():
    columna = row['Columna']
    variable = row['Variable']

    porcentajes, valores_normalizados = calcular_normalizado(df, columna, variable)

    if porcentajes is None or valores_normalizados is None:
        print(f"No hay datos para la combinación: {columna} - {variable}")
        continue

    resultados_list.append((columna, variable, porcentajes, valores_normalizados))

#graficar_resultados_multiple(resultados_list)

df_resultados_formateados = formatear_resultados(resultados_list)
df_resultados_formateados.to_csv('resultados_formateados.csv', index=True)
print(df_resultados_formateados)

País                                     CHIPRE    EGIPTO    ESPAÑA     MALTA  \
Índice                                                                          
People - yes                           0.428571  1.000000  0.786885  0.000000   
Role - primary                         0.500000  0.777778  0.745902  0.000000   
Role - secondary                       0.000000  0.000000  1.000000  0.000000   
Gender - male                          0.000000  1.000000  0.368852  0.000000   
Gender - female                        0.000000  0.000000  1.000000  0.000000   
Gender - other                              NaN       NaN       NaN       NaN   
Category - tourists                    0.500000  0.777778  0.918033  0.000000   
Category - locals                      0.000000  1.000000  0.000000  0.000000   
Category - workers                          NaN       NaN       NaN       NaN   
Age_Categories - children              0.000000  0.000000  0.688525  0.000000   
Age_Categories - young adult

In [124]:
resultados_totales = pd.concat(resultados_list, ignore_index=True)
print(resultados_totales)

TypeError: cannot concatenate object of type '<class 'tuple'>'; only Series and DataFrame objs are valid

In [73]:
resultados_totales.to_csv('resultados_normalizados_totales.csv', index=False)
print(resultados_totales.head())

  Columna Variable     País  Porcentaje  Valor Normalizado
0  People      yes   CHIPRE   14.285714           0.428571
1  People      yes   EGIPTO   33.333333           1.000000
2  People      yes   ESPAÑA   26.229508           0.786885
3  People      yes    MALTA    0.000000           0.000000
4  People      yes  TURQUIA   28.571429           0.857143
